## 準備

マルコフ決定過程 (MDP):  $\mathcal{M}=(\mathcal{S}, \mathcal{A}, \mathcal{P}, r, \rho, \gamma)$
* 状態空間:$\mathcal{S}$
* 行動空間:$\mathcal{A}$ 
* 状態遷移密度: $\mathcal{P}: \mathcal{S} \times \mathcal{A} \times \mathcal{S} \rightarrow \mathbb{R}_{+}$
* 報酬関数: $r: \mathcal{S} \times \mathcal{A} \rightarrow\left[-r_{\max }, r_{\max }\right]$
* 割引率: $\gamma \in(0,1)$
* $r_{\max} > 0$ 
* $\rho$ (初期状態分布)
* $H$ (軌跡長)




学習設定
* 方策: $\pi_\theta(\cdot \mid s)$  (where $\theta \in \mathbb{R}^d$ and $s \in \mathcal{S}$)
* 軌跡:

$$
\tau=\left(s_0, a_0, \cdots, s_{H-1}, a_{H-1}\right)
$$

* 軌跡の確率:

$$
p\left(\tau \mid \pi_\theta\right) \stackrel{\text { def }}{=} \rho\left(s_0\right) \pi_\theta\left(a_0 \mid s_0\right) \prod_{t=1}^{H-1} \mathcal{P}\left(s_t \mid s_{t-1}, a_{t-1}\right) \pi_\theta\left(a_t \mid s_t\right)
$$

* 期待収益(目的関数): $J(\theta) \stackrel{\text { def }}{=} \mathbb{E}_{\rho, \pi_\theta}\left[\sum_{t=0}^{+\infty} \gamma^t r\left(s_t, a_t\right)\right]$
* 有限期待収益: $J_H(\theta) \stackrel{\text { def }}{=} \mathbb{E}_{\rho, \pi_\theta}\left[\sum_{t=0}^{H-1} \gamma^t r\left(s_t, a_t\right)\right]$
* 期待収益(展開):総報酬(確率変数)と総報酬が得られる確率を用いる
$$
J(\theta) = \sum_{s_0 \in \mathcal{S}} \sum_{a_0 \in \mathcal{A}}  ... \left[ \sum_{t=0}^{\infty} \gamma^t r(s_t, a_t) \right]  P(s_0, a_0, s_1, a_1, ...)
$$
ここで、期待収益が得られる確率は、	
$$
P(s_0, a_0, s_1, a_1, ...) = \rho(s_0) \pi_\theta(a_0|s_0) P(s_1|s_0, a_0) \pi_\theta(a_1|s_1) P(s_2|s_1, a_1) ...
$$
であり、$p\left(\tau \mid \pi_\theta\right)$の無限版と解釈できる。
報酬関数は確率変数：状態および行動が$\rho$、$\pi$と$P$からサンプルされるので$s_t$、$a_t$が確率変数であるのだが、それを入力とする$r$はボレル可測関数であるため確率変数として扱える。また、確率変数の無限和は、有限値に収束する場合(割引無限とか)は、確率変数として扱えるので、総報酬も確率変数になる。


## 方策勾配とヘッセ行列

一次
* 有限期待収益の勾配:

$$
\nabla J_H(\theta)=\mathbb{E}_{\rho, \pi_\theta}[g(\tau, \theta)]
$$

	
なお、
$$
g(\tau, \theta)=\sum_{t=0}^{H-1}\left(\sum_{h=t}^{H-1} \gamma^h r\left(s_h, a_h\right)\right) \nabla \log \pi_\theta\left(a_t \mid s_t\right)
$$
である。
* 展開
有限期待収益は、軌跡の確率
$$J_H(\theta) = \mathbb{E}_{\rho, \pi_\theta}\left[\sum_{t=0}^{H-1} \gamma^t r(s_t, a_t)\right]$$
期待収益の期待値は、$p\left(\tau \mid \pi_\theta\right)$を用いて表せるので、
$$
\sum_\tau = \sum_{s_0 \in \mathcal{S}} \sum_{a_0 \in \mathcal{A}} \sum_{s_1 \in \mathcal{S}} \sum_{a_1 \in \mathcal{A}} ... \sum_{s_{H-1} \in \mathcal{S}} \sum_{a_{H-1} \in \mathcal{A}}
$$
とおくと、
$$
J_H(\theta) = \sum_{\tau} p(\tau|\pi_\theta) \left(\sum_{t=0}^{H-1} \gamma^t r(s_t, a_t)\right)
$$
と書ける。これを用いて、$\nabla J_H(\theta)$ を計算すると、

$$
\nabla J_H(\theta) = \sum_{\tau} \nabla p(\tau|\pi_\theta) \left(\sum_{t=0}^{H-1} \gamma^t r(s_t, a_t)\right)
$$
ここで、対数微分 $\nabla \log f(x)= \frac{ \nabla f(x)}{f(x)}$ を用いると、
$$
\nabla p(\tau|\pi_\theta) = p(\tau|\pi_\theta) \nabla \log p(\tau|\pi_\theta)
$$

$\nabla \log p(\tau|\pi_\theta)$について、

$$
\begin{aligned}
& \nabla \log p(\tau|\pi_\theta) \\
& = \nabla \log\left(\rho\left(s_0\right) \pi_\theta\left(a_0 \mid s_0\right) \prod_{t=1}^{H-1} P\left(s_t \mid s_{t-1}, a_{t-1}\right) \pi_\theta\left(a_t \mid s_t\right)\right) \\
& = \nabla \left[ \log \rho(s_0) + \sum_{t=0}^{H-1} \log \pi_\theta(a_t|s_t) + \sum_{t=0}^{H-1} \log P(s_{t+1}|s_t, a_t) \right] \\

\end{aligned}
$$
であり、$\nabla$は$\theta$に関しての勾配なので、右辺の第1項と第2項は0となり、
$$
\nabla \log p(\tau|\pi_\theta) = \sum_{t=0}^{H-1} \nabla \log \pi_\theta(a_t|s_t)
$$
よって、
$$
\nabla J_H(\theta) = \sum_{\tau} p(\tau|\pi_\theta) \left(\sum_{t=0}^{H-1} \nabla \log \pi_\theta(a_t|s_t)\right) \left(\sum_{t=0}^{H-1} \gamma^t r(s_t, a_t)\right)
$$

ここで、$g(\tau, \theta) = \left(\sum_{t=0}^{H-1} \nabla \log \pi_\theta(a_t|s_t)\right) \left(\sum_{t=0}^{H-1} \gamma^t r(s_t, a_t)\right)$ としたいが、$g$は、$\sum_{t=0}^{H-1} \gamma^t r(s_t, a_t)$ではなく、$\sum_{h=t}^{H-1} \gamma^h r\left(s_h, a_h\right)$である。$t$以降のみ考慮するのは、行動 $a_{t}$ が報酬に影響を与えるのは$t$以降だけだからである。

二次:
$$
\begin{aligned}
\nabla^2 J_H(\theta) & =\mathbb{E}_{\rho, \pi_\theta}[B(\tau, \theta)], \\
B(\tau, \theta) & \stackrel{\text { def }}{=} \nabla \Phi(\tau, \theta) \nabla \log p\left(\tau \mid \pi_\theta\right)^T+\nabla^2 \Phi(\tau, \theta) \\
\Phi(\tau, \theta) &\stackrel{\text { def }}{=} \sum_{t=0}^{H-1}\left(\sum_{h=t}^{H-1} \gamma^h r\left(s_h, a_h\right)\right) \log \pi_\theta\left(a_t \mid s_t\right)
\end{aligned}
$$






## 正規化モーメンタムを用いた方策勾配アルゴリズム(N-PG-IGT)
![[Pasted image 20250223003156.png]]

## ヘッセ行列を用いた再帰的方策勾配アルゴリズム((N)-HARPG)
![[Pasted image 20250223122217.png]]

### 収束解析
#### 解析のための準備
##### 仮定1(方策パラメータの正則性)
任意の$s, a \in \mathcal{S} \times \mathcal{A}$及び$\theta \in \mathbb{R}^d$に対して以下が成り立つと仮定する。
* 関数$\theta \mapsto \pi_\theta(a \mid s)$は、正かつ連続かつ二階微分まで微分可能
* 一次、二次勾配の大きさは有限($\left\|\nabla \log \pi_\theta(a \mid s)\right\| \leq M_g$、$\left\|\nabla^2 \log \pi_\theta(a \mid s)\right\| \leq M_h$を満たすような$M_g>0$、$M_h>0$が存在する)
##### 命題1
* 関数$J$ は$L_g$-smoothかつ $L_g=\frac{r_{\max }\left(M_g^2+M_h\right)}{(1-\gamma)^2}$.
* $\mathbb{E}\left[\left\|g(\tau \mid \theta)-\nabla J_H(\theta)\right\|^2\right] \leq \sigma_g^2$ かつ $\sigma_g^2=\frac{r_{\max }^2 M_g^2}{(1-\gamma)^3}$.
	MSE?
$$
\begin{aligned}
\mathbb{E}\left[\left\|g(\tau \mid \theta)-\nabla J_H(\theta)\right\|^2\right] &= \mathbb{E}\left[\left(g(\tau \mid \theta)-\nabla J_H(\theta)\right) \cdot \left(g(\tau \mid \theta)-\nabla J_H(\theta)\right)\right]\\
&= \mathbb{E}\left[\|g(\tau \mid \theta)\|^2 - 2g(\tau \mid \theta) \cdot \nabla J_H(\theta) + \|\nabla J_H(\theta)\|^2\right]\\
&= \mathbb{E}\left[\|g(\tau \mid \theta)\|^2\right] - 2\mathbb{E}\left[g(\tau \mid \theta)\right] \cdot \nabla J_H(\theta) + \mathbb{E}\left[\|\nabla J_H(\theta)\|^2\right]\\
&= \mathbb{E}\left[\|g(\tau \mid \theta)\|^2\right] - 2\nabla J_H(\theta) \cdot \nabla J_H(\theta) + \|\nabla J_H(\theta)\|^2\\
&= \mathbb{E}\left[\|g(\tau \mid \theta)\|^2\right] - \|\nabla J_H(\theta)\|^2
\end{aligned}
$$
* $\mathbb{E}\left[\left\|B(\tau \mid \theta)-\nabla^2 J_H(\theta)\right\|^2\right] \leq \sigma_h^2$ かつ $\sigma_h^2=$ $\frac{r_{\max }^2\left(H^2 M_g^4+M_h^2\right)}{(1-\gamma)^4}$ (一つ上の式が成立したときに成立)
##### 仮定2(方策とその二次対数微分は$L$-smooth)
**仮定の仮定**
* 任意の$s, a \in \mathcal{S} \times \mathcal{A}$及び$\theta \in \mathbb{R}^d$に対して以下が成り立つ
* 関数$\theta \mapsto \pi_\theta(a \mid s)$は、正かつ連続かつ二階微分まで微分可能
任意の$\theta,\theta'$について、以下を満たす任意の$l_2 > 0$が存在する。
$$
\left\|\nabla^2 \log \pi_\theta(a \mid s)-\nabla^2 \log \pi_{\theta^{\prime}}(a \mid s)\right\| \leq l_2\left\|\theta-\theta^{\prime}\right\|
$$
##### 補題1(期待収益は$L_h$-リプシッツ連続)
仮定1と仮定2が成立するとする。
任意の$\theta,\theta'$について、
$$
\theta, \theta^{\prime} \in \mathbb{R}^d,\left\|\nabla^2 J(\theta)-\nabla^2 J\left(\theta^{\prime}\right)\right\| \leq L_h\left\|\theta-\theta^{\prime}\right\|
$$
であり、$L_h=\mathcal{O}\left((1-\gamma)^{-3}\right)$である。

##### 仮定3(Fisher-non-degenerate parameterized policy)

**定義:$\pi$により誘発されたフィッシャー情報行列**
$$
F_\rho(\theta) \stackrel{\text { def }}{=} \mathbb{E}_{s \sim d_\rho^{\pi_\theta}, a \sim \pi_\theta(\cdot \mid s)}\left[\nabla \log \pi_\theta(a \mid s) \nabla \log \pi_\theta(a \mid s)^{\top}\right]
$$
なお、
$$
d_\rho^{\pi_\theta}(\cdot) \stackrel{\text { def }}{=}(1-\gamma) \sum_{t=0}^{\infty} \gamma^t \mathbb{P}_{\rho, \pi_\theta}\left(s_t \in \cdot\right)
$$
**展開**
$$
F_\rho(\theta) \stackrel{\text { def }}{=}\sum_{s \in \mathcal{S}} d_\rho^{\pi_\theta}(s) \sum_{a \in \mathcal{A}} \pi_\theta(a|s) \nabla \log \pi_\theta(a | s) \nabla \log \pi_\theta(a | s)^{\top}
$$

##### 仮定3(Fisher-non-degenerate policy)

全ての$\theta$に対して、$F_\rho(\theta) \succcurlyeq \mu_F \cdot I_d$になるような固有値$\mu_F > 0$が存在する。
* 逆行列計算できる

##### 仮定4
全ての$\theta$について、trasnfer errorが以下になるような、$\epsilon_{\text{bias}} \geq 0$が存在する
$$
\mathbb{E}\left[\left(A^{\pi_\theta}(s, a)-(1-\gamma) w^*(\theta)^{\top} \nabla \log \pi_\theta(a \mid s)\right)^2\right] \leq \varepsilon_{\text {bias }}
$$
*  **$A^{\pi_\theta}(s, a)$**: アドバンテージ関数 
$$
A^\pi(s, a) \stackrel{\text { def }}{=} Q^\pi(s, a)-V^\pi(s)
$$ 

$\pi_{\theta}$が$A^{\pi_\theta}(s, a)$をいい感じに$(1-\gamma) w^*(\theta)^{\top} \nabla \log \pi_\theta(a \mid s))^2$で近似できる

##### 補題2(Relaxed弱いGradient domination)
仮定1、仮定3、仮定４が成立する場合、
$$
\forall \theta \in \mathbb{R}^d, \quad \varepsilon^{\prime}+\|\nabla J(\theta)\| \geq \sqrt{2 \mu}\left(J^*-J(\theta)\right)
$$
なお、

$$
\text { where } \varepsilon^{\prime}=\frac{\mu_F \sqrt{\varepsilon_{\text {bias }}}}{M_o(1-\gamma)} \text { and } \mu=\frac{\mu_F^2}{2 M^2}
$$


#### 補題4

仮定1が満たされているとする。このとき、任意の $\theta \in \mathbb{R}^d$ と任意の $H \geq 1$ に対して、以下が成り立つ。

$$
\|\nabla J_H(\theta) - \nabla J(\theta)\| \leq D_g \gamma^H, \quad \|\nabla^2 J_H(\theta) - \nabla^2 J(\theta)\| \leq D_h \gamma^H
$$

ここで、$D_g = \frac{M_g r_{\max}}{1-\gamma} \sqrt{\frac{1}{1-\gamma} + H}$,  $D_h = \frac{r_{\max}(M_h + M_g^2)}{1-\gamma} (H + \frac{1}{1-\gamma})$ である。

#### 補題12
$a$ を正の実数、$\tau$ を正の整数とし、非負の系列 $\{r_t\}_{t \ge 0}$ が任意の整数 $t \ge 0$ に対して、

$$
r_{t+1} - r_t \le -a \alpha_t r_t + \beta_t
$$

を満たすと仮定する。ここで、$\{\alpha_t\}_{t \ge 0}$ と $\{\beta_t\}_{t \ge 0}$ は非負の系列であり、任意の $t$ に対して $a \alpha_t \le 1$ が成り立つ。このとき、$\alpha_t = \frac{2}{a(t+\tau)}$ とすると、任意の整数 $t_0, T \ge 1$ に対して、以下が成り立つ。

$$
r_T \le \frac{(t_0 + \tau - 1)^2 r_{t_0}}{(T + \tau - 1)^2} + \frac{\sum_{t=0}^{T-1} \beta_t (t + \tau)^2}{(T + \tau - 1)^2}
$$

#### 補題14
$q \in [0, 1]$ とし、任意の整数 $t$ に対して $\eta_t = \left(\frac{2}{t+2}\right)^q$ とする。このとき、任意の整数 $t$ と任意の整数 $T \geq 1$ に対して、以下が成り立つ。

$$
\eta_t (1 - \eta_{t+1}) \leq \eta_{t+1} \\
\prod_{t=0}^{T-1} (1 - \eta_{t+1}) \leq \eta_T
$$


#### 補題15
$q \in [0, 1)$, $p \geq 0$, $\gamma_0 > 0$ とし、任意の整数 $t$ に対して $\eta_t = \left(\frac{2}{t+2}\right)^q$, $\gamma_t = \gamma_0 \left(\frac{2}{t+2}\right)^p$ とする。このとき、任意の整数 $t$ と任意の整数 $T \geq 1$ に対して、以下が成り立つ。

$$
\sum_{t=0}^{T-1} \gamma_t \prod_{\tau=t+1}^{T-1} (1 - \eta_\tau) \leq C \gamma_T \eta_T^{-1}
$$

ここで、

$$
C = C(p, q) = 2^{p-q}(1-q)^{-1} t_0 \exp(2^q(1-q) t_0^{1-q}) + 2^{2p+1-q}(1-q)^{-2}
$$

および

$$
t_0 = \max \left\{ \left( \frac{p}{(1-q)2^q} \right)^{\frac{1}{1-q}}, 2\left( \frac{p-q}{(1-q)^2} \right) \right\}^{\frac{1}{1-q}}
$$

とする。

#### 定理1(N-PG-IGTのグローバル収束)
仮定1、仮定2、仮定3、仮定4が成立とする。$\gamma_t=\frac{6 M_g}{\mu_F(t+2)}, \eta_t=\left(\frac{2}{t+2}\right)^{4 / 5}$、$\eta_t=\left(\frac{2}{t+2}\right)^{4 / 5}$とし、$H=(1-\gamma)^{-1} \log (T+1)$とする。
このとき、任意の$T \geq 1$について、以下が成り立つ。
$$
J^*-\mathbb{E}\left[J\left(\theta_T\right)\right] \leq \mathcal{O}\left(\frac{\sigma_g+L_h}{(T+1)^{2 / 5}}\right)+\frac{\sqrt{\varepsilon_{\text {bias }}}}{1-\gamma}
$$
#### 定理1の証明







### 定理証明の流れ

1.パラメータ更新上昇補題(**補題5**)を示す。$\theta_{t+1}$におけるシミュレーション誤差の上界を再帰的に$\theta_t$をもちいて表現できる。
* 更新すればするほど、誤差上界が小さくなっていくことが言える。
2.次に、緩和された弱いGradient dominanceを用いた不等式(**補題6**)を示す。(**サンプル複雑度の向上に寄与**)
3.分散誤差を制御する不等式の導出(**補題**7)をする(**サンプル複雑度の向上に寄与**)

#### 補題5(パラメータ更新上昇補題)
仮定 1 が成り立つとする。  $\left(d_t\right)$ を $\mathbb{R}^d$ 内の任意のベクトル列とし、以下で定義される列 $\left(\theta_t\right)$ を考える。

$$
\theta_{t+1}=\theta_t+\gamma_t \frac{d_t}{\left\|d_t\right\|}
$$

ここで、$\theta_0 \in \mathbb{R}^d$ とする（$d_t=0$ の場合は $\theta_{t+1}=\theta_t$）。このとき、任意の正のステップサイズ $\gamma_t$ に対して、すべての整数 $t \geq 1$ について、以下が成り立つ。

$$
-J\left(\theta_{t+1}\right) \leq-J\left(\theta_t\right)-\frac{\gamma_t}{3}\left\|\nabla J\left(\theta_t\right)\right\|+\frac{8 \gamma_t}{3}\left\|d_t-\nabla J_H\left(\theta_t\right)\right\|+\frac{L_g \gamma_t^2}{2}+\frac{4 \gamma_t}{3}\left\|\nabla J_H\left(\theta_t\right)-\nabla J\left(\theta_t\right)\right\|
$$

#### 補題5の証明
Descent Lemma(Beck本参照)を用いると、以下のようになる。
$$
\begin{aligned}
-J\left(\theta_{t+1}\right) & \leq-J\left(\theta_t\right)-\left\langle\nabla J\left(\theta_t\right), \theta_{t+1}-\theta_t\right\rangle+\frac{L_g}{2}\left\|\theta_{t+1}-\theta_t\right\|^2 \\
& =-J\left(\theta_t\right)-\gamma_t \frac{\left\langle\nabla J\left(\theta_t\right), d_t\right\rangle}{\left\|d_t\right\|}+\frac{L_g \gamma_t^2}{2} \\
& \leq-J\left(\theta_t\right)-\gamma_t \frac{\left\langle\nabla J_H\left(\theta_t\right), d_t\right\rangle}{\left\|d_t\right\|}+\frac{L_g \gamma_t^2}{2}+\gamma_t\left\|\nabla J_H\left(\theta_t\right)-\nabla J\left(\theta_t\right)\right\| .
\end{aligned}
$$
二番目、三番目のステップでは、$\theta_{t+1}-\theta_t=\gamma_t\frac{d_t}{\left\|d_t\right\|}\leq \gamma_t$を用いている。
次に、右辺第二項をバウンドすることを考える。$\hat{e}_t=d_t-\nabla J_H\left(\theta_t\right)$と定義する。
* $\left\|\hat{e}_t\right\| \leq \frac{1}{2}\left\|\nabla J_H\left(\theta_t\right)\right\|$のとき、
$$
\begin{aligned}
-\frac{\left\langle\nabla J_H\left(\theta_t\right), d_t\right\rangle}{\left\|d_t\right\|} & =\frac{-\left\|\nabla J_H\left(\theta_t\right)\right\|^2-\left\langle\nabla J_H\left(\theta_t\right), \hat{e}_t\right\rangle}{\left\|d_t\right\|} \\
& \leq \frac{-\left\|\nabla J_H\left(\theta_t\right)\right\|^2+\left\|\nabla J_H\left(\theta_t\right)\right\|\left\|\hat{e}_t\right\|}{\left\|d_t\right\|} \\
& \leq \frac{-\left\|\nabla J_H\left(\theta_t\right)\right\|^2+\frac{1}{2}\left\|\nabla J_H\left(\theta_t\right)\right\|^2}{\left\|d_t\right\|} \\
& \leq-\frac{\left\|\nabla J_H\left(\theta_t\right)\right\|^2}{2\left(\left\|\nabla J_H\left(\theta_t\right)\right\|+\left\|\hat{e}_t\right\|\right)} \\
& \leq-\frac{1}{3}\left\|\nabla J_H\left(\theta_t\right)\right\|
\end{aligned}
$$
ここで、最初の不等式は、コーシーシュワルツの不等式($|<A,B>|\leq||A||||B||$)を使っている。二番目の不等式は、パターンの条件の上界を用いている。最後にかけての不等式は、定義の代入で導出できる。

* $\left\|\hat{e}_t\right\| \geq \frac{1}{2}\left\|\nabla J_H\left(\theta_t\right)\right\|$のとき、

**当たり前tips**
* 式に値の上界を代入して上界不等式→OK
* 式に値の下界を代入して上界不等式→大小関係を考え直さない解けない

コーシーシュワルツの不等式より、
$$
\left|\left\langle\nabla J_H\left(\theta_t\right), d_t\right\rangle\right| \leq\left\|\nabla J_H\left(\theta_t\right)\right\|\left\|d_t\right\|
$$
であり、両者を$\left\|d_t\right\|$で割ると、
$$
-\left\|\nabla J_H\left(\theta_t\right)\right\| \leq \frac{\left\langle\nabla J_H\left(\theta_t\right), d_t\right\rangle}{\left\|d_t\right\|} \leq\left\|\nabla J_H\left(\theta_t\right)\right\|
$$
であり、自明に
$$
-\frac{\left\langle\nabla J_H\left(\theta_t\right), d_t\right\rangle}{\left\|d_t\right\|}  \leq\left\|\nabla J_H\left(\theta_t\right)\right\| 
$$
が成立する。よって、
$$
\begin{aligned}
-\frac{\left\langle\nabla J_H\left(\theta_t\right), d_t\right\rangle}{\left\|d_t\right\|} & \leq\left\|\nabla J_H\left(\theta_t\right)\right\| \\
& =-\frac{1}{3}\left\|\nabla J_H\left(\theta_t\right)\right\|+\frac{4}{3}\left\|\nabla J_H\left(\theta_t\right)\right\| \\
& \leq-\frac{1}{3}\left\|\nabla J_H\left(\theta_t\right)\right\|+\frac{8}{3}\left\|\hat{e}_t\right\|
\end{aligned}
$$
ここで、不等式は$\left\|\hat{e}_t\right\| \geq \frac{1}{2}\left\|\nabla J_H\left(\theta_t\right)\right\|$の代入で導出できる。
最後に、2パターンをまとめて、
$$
\begin{aligned}
-J\left(\theta_{t+1}\right) & \leq-J\left(\theta_t\right)-\frac{\gamma_t}{3}\left\|\nabla J_H\left(\theta_t\right)\right\|+\frac{8 \gamma_t}{3}\left\|\hat{e}_t\right\|+\frac{L_g \gamma_t^2}{2}+\gamma_t\left\|\nabla J_H\left(\theta_t\right)-\nabla J\left(\theta_t\right)\right\| \\
& \leq-J\left(\theta_t\right)-\frac{\gamma_t}{3}\left\|\nabla J\left(\theta_t\right)\right\|+\frac{8 \gamma_t}{3}\left\|\hat{e}_t\right\|+\frac{L_g \gamma_t^2}{2}+\frac{4 \gamma_t}{3}\left\|\nabla J_H\left(\theta_t\right)-\nabla J\left(\theta_t\right)\right\|
\end{aligned}
$$
となる。

#### 補題6(緩和された弱いgradient dominance)
仮定 1、3、および 4 が満たされているとする。補題 5(上昇補題) の設定において、任意の正のステップサイズ $\gamma_t$ に対して、すべての整数 $t \geq 1$ について、以下が成り立つ。

$$
\delta_{t+1}-\delta_t \leq-\frac{\sqrt{2 \mu} \gamma_t}{3} \delta_t+\frac{8 \gamma_t}{3} \mathbb{E}\left[\left\|d_t-\nabla J_H\left(\theta_t\right)\right\|\right]+\frac{L_g \gamma_t^2}{2}+\frac{\varepsilon^{\prime} \gamma_t}{3}+\frac{4}{3} \gamma_t D_g \gamma^H
$$

#### 補題6の証明
補題2(緩和された弱いgradient dominance)と、補題5を組み合わせる。
補題5は以下の不等式が成立する。
$$
-J\left(\theta_{t+1}\right) \leq-J\left(\theta_t\right)-\frac{\gamma_t}{3}\left\|\nabla J\left(\theta_t\right)\right\|+\frac{8 \gamma_t}{3}\left\|d_t-\nabla J_H\left(\theta_t\right)\right\|+\frac{L_g \gamma_t^2}{2}+\frac{4 \gamma_t}{3}\left\|\nabla J_H\left(\theta_t\right)-\nabla J\left(\theta_t\right)\right\|
$$
* 右辺第二項について補題2、最後の項について補題4を用いて、
$$
-J\left(\theta_{t+1}\right) \leq-J\left(\theta_t\right)-\frac{\sqrt{2 \mu} \gamma_t}{3}\left(J^{\star}-J\left(\theta_t\right)\right)+\frac{8 \gamma_t}{3}\left\|d_t-\nabla J_H\left(\theta_t\right)\right\|+\frac{L_g \gamma_t^2}{2}+\frac{\varepsilon^{\prime} \gamma_t}{3}+\frac{4}{3} \gamma_t D_g \gamma^H
$$
となる。$J^*$を両方の項にかけると、補題6と同じになる。
**最後の項**がサンプル効率上げる工夫っぽい？

#### 補題7(分散誤差制御)
仮定 1 と 2 が満たされているとする。 $\eta_t=\left(\frac{2}{t+2}\right)^{4 / 5}$ および $\gamma_t=\frac{6}{\sqrt{2 \mu}(t+2)}$ として、N-PG-IGTによって生成される列 $\left(\tilde{\theta}_t\right)$、$\left(d_t\right)$、および $\left(\theta_t\right)$ を考える。このとき、任意の整数 $t \geq 0$ について、以下が成り立つ。

$$
\begin{aligned}
& \mathbb{E}\left[\left\|d_t-\nabla J_H\left(\theta_t\right)\right\|\right] \leq 2 \sqrt{C\left(\frac{8}{5}, \frac{4}{5}\right)} \sigma_g \eta_t^{1 / 2}+C\left(\frac{6}{5}, \frac{4}{5}\right) L_h \gamma_t^2 \eta_t^{-2} \\
&+2 D_g \gamma^H C\left(0, \frac{4}{5}\right) \eta_t^{-1}+D_h \gamma^H C\left(1, \frac{4}{5}\right) \gamma_t \eta_t^{-1}
\end{aligned}
$$

ここで、$q \in[0,1)$、$p>0$ に対する $C(p, q)$ は補題 15 で定義される定数。$\sigma$、つまり分散で誤差上界を抑えることが可能っぽい。分散の

#### 補題7の証明
##### ステップ1：$\hat{e}_t$の変形

以下のように変数を定義する。$S_t,Z_t$はサンプルから得た近似、$\bar{S_t},\bar{Z_t}$は真値を用いている。
$$
\begin{aligned}
\hat{e}_t& \stackrel{\text { def }}{=} d_t-\nabla J_H\left(\theta_t\right)\\ e_t & \stackrel{\text { def }}{=} g\left(\tilde{\tau}_t, \tilde{\theta}_t\right)-\nabla J_H\left(\tilde{\theta}_t\right) \\
S_t & \stackrel{\text { def }}{=} \nabla J_H\left(\theta_{t-1}\right)-\nabla J_H\left(\theta_t\right)+\nabla^2 J_H\left(\theta_t\right)\left(\theta_{t-1}-\theta_t\right), \\
\bar{S}_t & \stackrel{\text { def }}{=} \nabla J\left(\theta_{t-1}\right)-\nabla J\left(\theta_t\right)+\nabla^2 J\left(\theta_t\right)\left(\theta_{t-1}-\theta_t\right) \\
Z_t & \stackrel{\text { def }}{=} \nabla J_H\left(\tilde{\theta}_t\right)-\nabla J_H\left(\theta_t\right)+\nabla^2 J_H\left(\theta_t\right)\left(\tilde{\theta}_t-\theta_t\right)\\
\bar{Z}_t& \stackrel {\text { def }}{=} \nabla J\left(\tilde{\theta}_t\right)-\nabla J\left(\theta_t\right)+\nabla^2 J\left(\theta_t\right)\left(\tilde{\theta}_t-\theta_t\right) 
\end{aligned}
$$
とする。補題1と補題4を使って、$S_t,Z_t$のノルム、$\bar{S_t},\bar{Z_t}$との誤差を図る。この定義は式の簡略化に使う。
$$
\begin{aligned}
\left\|S_t\right\| & \leq L_h\left\|\theta_t-\theta_{t-1}\right\|^2+\left\|S_t-\bar{S}_t\right\|=L_h \gamma_{t-1}^2+\left\|S_t-\bar{S}_t\right\| \\
\left\|S_t-\bar{S}_t\right\| & \leq 2 D_g \gamma^H+D_h \gamma^H \gamma_{t-1} \\
\left\|Z_t\right\| & \leq L_h\left\|\tilde{\theta}_t-\theta_t\right\|^2+\left\|Z_t-\bar{Z}_t\right\|=L_h \frac{\left(1-\eta_t\right)^2}{\eta_t^2}\left\|\theta_t-\theta_{t-1}\right\|^2+\left\|Z_t-\bar{Z}_t\right\|, \\
\left\|Z_t-\bar{Z}_t\right\| & \leq 2 D_g \gamma^H+D_h \gamma^H \frac{\gamma_{t-1}}{\eta_t} .
\end{aligned}
$$
さて、今回バウンドしたいものは、$\hat{e}_t=  d_t-\nabla J_H\left(\theta_t\right)$である。式変形を以下のように行う。式変形は愚直に行えば導出できる(た)。
$$
\begin{aligned}
&\begin{aligned}
\hat{e}_t= & d_t-\nabla J_H\left(\theta_t\right) \\
\stackrel{(i)}{=} & \left(1-\eta_t\right) d_{t-1}+\eta_t g\left(\tilde{\tau}_t, \tilde{\theta}_t\right)-\nabla J_H\left(\theta_t\right) \\
= & \left(1-\eta_t\right)\left(\hat{e}_{t-1}+\nabla J_H\left(\theta_{t-1}\right)\right)+\eta_t g\left(\tilde{\tau}_t, \tilde{\theta}_t\right)-\nabla J_H\left(\theta_t\right) \\
= & \left(1-\eta_t\right) \hat{e}_{t-1}+\eta_t e_t+\left(1-\eta_t\right)\left(\nabla J_H\left(\theta_{t-1}\right)-\nabla J_H\left(\theta_t\right)\right)+\eta_t\left(\nabla J_H\left(\tilde{\theta}_t\right)-\nabla J_H\left(\theta_t\right)\right) \\
\stackrel{(i i)}{=} & \left(1-\eta_t\right) \hat{e}_{t-1}+\eta_t e_t+\left(1-\eta_t\right)\left(\nabla J_H\left(\theta_{t-1}\right)-\nabla J_H\left(\theta_t\right)+\nabla^2 J_H\left(\theta_t\right)\left(\theta_{t-1}-\theta_t\right)\right) \\
& +\eta_t\left(\nabla J_H\left(\tilde{\theta}_t\right)-\nabla J_H\left(\theta_t\right)+\nabla^2 J_H\left(\theta_t\right)\left(\tilde{\theta}_t-\theta_t\right)\right) \\
& \quad-\left(1-\eta_t\right) \nabla^2 J_H\left(\theta_t\right)\left(\theta_{t-1}-\theta_t\right)-\eta_t \nabla^2 J_H\left(\theta_t\right)\left(\tilde{\theta}_t-\theta_t\right)
\end{aligned}\\
&\stackrel{(i i i)}{=}\left(1-\eta_t\right) \hat{e}_{t-1}+\eta_t e_t+\left(1-\eta_t\right) S_t+\eta_t Z_t,
\end{aligned}
$$
$(i)$は、$d_t = (1 - \eta_t) d_{t-1} + \eta_t g(\tilde{\tau}_t, \tilde{\theta}_t)$を用いて代入するだけである。
$(ii)$は、三、四番目の項にヘッセ行列の項を追加している。具体的には、二番目の項には、
$$
\nabla^2 J_H\left(\theta_t\right)\left(\theta_{t-1}-\theta_t\right)
$$
三番目の項には、
$$
\nabla^2 J_H\left(\theta_t\right)\left(\tilde{\theta}_t-\theta_t\right)
$$
を追加している。また、最終二項
$$
-\left(1-\eta_t\right) \nabla^2 J_H\left(\theta_t\right)\left(\theta_{t-1}-\theta_t\right)-\eta_t \nabla^2 J_H\left(\theta_t\right)\left(\tilde{\theta}_t-\theta_t\right)
$$
で追加した分をすべて引いている。
**サンプル効率向上ポイント?：$\hat{e_t}$の更新式における最終二項の相殺**
$\tilde{\theta}_t = \theta_t + \frac{1 - \eta_t}{\eta_t} (\theta_t - \theta_{t-1})$という定義から、式変形を行うと、
$$
η_t (\tilde{θ}_t - θ_t) = (1 - η_t)(θ_t - θ_{t-1})
$$
となる。両辺に$-∇²J_H(θ_t)$を掛けると、
$$
-η_t ∇²J_H(θ_t)(\tilde{θ}_t - θ_t) = -(1 - η_t) ∇²J_H(θ_t)(θ_t - θ_{t-1})
$$
となる。さらに、補題5、上昇補題($\theta_{t+1}=\theta_t+\gamma_t \frac{d_t}{\left\|d_t\right\|}$)より、$\theta_t\geq\theta_{t-1}$がすべての$t$に対して成り立つので、右辺を変換すると、
$$
-η_t ∇²J_H(θ_t)(\tilde{θ}_t - θ_t) = (1 - η_t) ∇²J_H(θ_t)(θ_{t-1}-θ_t )
$$
となり、最終二項は
$$
-\left(1-\eta_t\right) \nabla^2 J_H\left(\theta_t\right)\left(\theta_{t-1}-\theta_t\right)+(1 - η_t) ∇²J_H(θ_t)(θ_{t-1}-θ_t )=0
$$
となり消える。ヘッセ行列が消えるのでアルゴリズムの計算量減りそう。
##### ステップ2：$\hat{e}_t$のバウンド
Momentumの減衰を以下のように定義する。
$$
\zeta_{t+1, T} = \prod_{\tau=t+1}^{T-1} (1 - \eta_{\tau+1})
$$
これは、$t$が進につれて、過去の勾配の影響が指数関数的に減衰することを示している。
次に、ステップ1で得られた結果を使って式変形を行う。
$$
\hat{e}_t = (1-\eta_t) \hat{e}_{t-1} + \eta_t e_t + (1-\eta_t)S_t + \eta_t Z_t
$$
例として、$t = T-1$ の場合を考える。

$$
\hat{e}_{T-1} = (1-\eta_{T-1})\hat{e}_{T-2} + \eta_{T-1}e_{T-1} + (1-\eta_{T-1})S_{T-1} + \eta_{T-1}Z_{T-1}
$$
であり、$\hat{e}_T$に代入すると、
$$
\begin{aligned}
\hat{e}_T &= (1-\eta_T)\left[ (1-\eta_{T-1})\hat{e}_{T-2} + \eta_{T-1}e_{T-1} + (1-\eta_{T-1})S_{T-1} + \eta_{T-1}Z_{T-1} \right] \\
&+ \eta_T e_T + (1-\eta_T)S_T + \eta_T Z_T
\end{aligned}
$$
となる。ここで、Momentumの減衰を用いて式を書き直すと、
$$
\hat{e}_T=\zeta_{0, T} \hat{e}_0+\sum_{t=0}^{T-1} \eta_{t+1} \zeta_{t+1, T} e_{t+1}+\sum_{t=0}^{T-1}\left(1-\eta_{t+1}\right) \zeta_{t+1, T} S_{t+1}+\sum_{t=0}^{T-1} \eta_{t+1} \zeta_{t+1, T} Z_{t+1}
$$
となる。ここで、
$$
\mathcal{F}_t \stackrel{\text { def }}{=} \sigma\left(\left\{\tilde{\theta}_0, \tilde{\tau}_0, \ldots, \tilde{\tau}_{t-1}\right\}\right)
$$
を考える。このとき、$e_t$ はマルチンゲール差分列になり、
$$
\mathbb{E}\left[\left\langle e_{t_1}, e_{t_2}\right\rangle\right]=\mathbb{E}\left[\mathbb{E}\left[\left\langle e_{t_1}, e_{t_2}\right\rangle \mid \mathcal{F}_{t_2}\right]\right]=\mathbb{E}\left[\left\langle e_{t_1}, \mathbb{E}\left[e_{t_2} \mid \mathcal{F}_{t_2}\right]\right\rangle\right]=0
$$
が成立する。よって、以下のように式変形が行える。式変形は代入等で計算できる。
$$
\begin{aligned}
& \mathbb{E}\left[\left\|\hat{e}_T\right\|\right] \leq \zeta_{0, T} \mathbb{E}\left[\left\|\hat{e}_0\right\|\right]+\mathbb{E}\left[\left\|\sum_{t=0}^{T-1} \eta_{t+1} \zeta_{t+1, T} e_{t+1}\right\|\right] \\
&\left.+\mathbb{E}\left[\left\|\sum_{t=0}^{T-1}\left(1-\eta_{t+1}\right) \zeta_{t+1, T} S_{t+1}\right\|\right]+\mathbb{E}\left[\left\|\sum_{t=0}^{T-1} \eta_{t+1} \zeta_{t+1, T} Z_{t+1}\right\|\right]\right] \\
& \leq \zeta_{0, T} \sigma_g+\left(\mathbb{E}\left[\left\|\sum_{t=0}^{T-1} \eta_{t+1} \zeta_{t+1, T} e_{t+1}\right\|^2\right]\right)^{1 / 2} \\
&+\sum_{t=0}^{T-1}\left(1-\eta_{t+1}\right) \zeta_{t+1, T} \mathbb{E}\left[\left\|S_{t+1}\right\|\right]+\sum_{t=0}^{T-1} \eta_{t+1} \zeta_{t+1, T} \mathbb{E}\left[\left\|Z_{t+1}\right\|\right] \\
& \leq \zeta_{0, T} \sigma_g+\left(\sum_{t=0}^{T-1} \eta_{t+1}^2 \zeta_{t+1, T}^2 \mathbb{E}\left[\left\|e_{t+1}\right\|^2\right]\right)^{1 / 2} \\
&+L_h \sum_{t=0}^{T-1}\left(1-\eta_{t+1}\right) \zeta_{t+1, T} \gamma_t^2+L_h \sum_{t=0}^{T-1} \frac{\gamma_t^2}{\eta_{t+1}} \zeta_{t+1, T} \\
&+\sum_{t=0}^{T-1}\left(1-\eta_{t+1}\right) \zeta_{t+1, T}\left\|S_{t+1}-\bar{S}_{t+1}\right\|+\sum_{t=0}^{T-1} \eta_{t+1} \zeta_{t+1, T}\left\|Z_{t+1}-\bar{Z}_{t+1}\right\|
\end{aligned}
$$

と変形できる。補題14、15を用いると、
$$
\begin{aligned}
\mathbb{E}\left[\left\|\hat{e}_T\right\|\right] \leq & \eta_T \sigma_g+\sqrt{C\left(\frac{8}{5}, \frac{4}{5}\right)} \eta_T^{1 / 2} \sigma_g+C\left(\frac{6}{5}, \frac{4}{5}\right) \gamma_T^2 \eta_T^{-2} L_h \\
& +2 D_g \gamma^H C\left(0, \frac{4}{5}\right) \eta_T^{-1}+D_h \gamma^H C\left(1, \frac{4}{5}\right) \gamma_T \eta_T^{-1} \\
\leq & 2 \cdot \sqrt{C\left(\frac{8}{5}, \frac{4}{5}\right)} \eta_T^{1 / 2} \sigma_g+C\left(\frac{6}{5}, \frac{4}{5}\right) \gamma_T^2 \eta_T^{-2} L_h \\
& +2 D_g \gamma^H C\left(0, \frac{4}{5}\right) \eta_T^{-1}+D_h \gamma^H C\left(1, \frac{4}{5}\right) \gamma_T \eta_T^{-1}
\end{aligned}
$$
#### 結合
補題6、７を用いると、
$$
\begin{aligned}
\delta_{t+1} \leq & \left(1-\frac{\sqrt{2 \mu} \gamma_t}{3}\right) \delta_t+\frac{8 \gamma_t}{3} \mathbb{E}\left[\left\|d_t-\nabla J_H\left(\theta_t\right)\right\|\right]+\frac{L_g \gamma_t^2}{2}+\frac{\gamma_t \varepsilon^{\prime}}{3}+\frac{4 \gamma_t D_g}{3} \gamma^H \\
\leq & \left(1-\frac{\sqrt{2 \mu} \gamma_t}{3}\right) \delta_t+\frac{16}{3} \sqrt{C\left(\frac{8}{5}, \frac{4}{5}\right)} \sigma_g \gamma_t \eta_t^{1 / 2}+\frac{8}{3} C\left(\frac{6}{5}, \frac{4}{5}\right) L_h \gamma_t^3 \eta_t^{-2}+\frac{L_g \gamma_t^2}{2}+\frac{\gamma_t \varepsilon^{\prime}}{3}+\frac{4 \gamma_t D_g}{3} \gamma^H \\
& \quad+\frac{16}{3} D_g \gamma^H C\left(0, \frac{4}{5}\right) \gamma_t \eta_t^{-1}+\frac{8}{3} D_h \gamma^H C\left(1, \frac{4}{5}\right) \gamma_t^2 \eta_t^{-1} .
\end{aligned}
$$
補題12と以下の代入を用いて、
$$
\begin{aligned}
&\alpha_t=\gamma_t, a=\frac{3}{\sqrt{2 \mu}}, t_0=0, \tau=2, \beta_t=\frac{16}{3} \sqrt{C\left(\frac{8}{5}, \frac{4}{5}\right)} \sigma_g \gamma_t \eta_t^{1 / 2}+\frac{8}{3} C\left(\frac{6}{5}, \frac{4}{5}\right) L_h \gamma_t^3 \eta_t^{-2}+\frac{L_g \gamma_t^2}{2}+ \\
& \frac{\gamma_t \varepsilon^{\prime}}{3}+\frac{4 \gamma_t D_g}{3} \gamma^H+\frac{16}{3} D_g \gamma^H C\left(0, \frac{4}{5}\right) \gamma_t \eta_t^{-1}+\frac{8}{3} D_h \gamma^H C\left(1, \frac{4}{5}\right) \gamma_t^2 \eta_t^{-1}
\end{aligned}
$$

$$
\begin{aligned}
\delta_T \leq & \frac{\delta_0}{(T+1)^2}+\frac{\sum_{t=0}^{T-1} \beta_t(t+2)^2}{(T+1)^2} \\
\leq & \frac{\delta_0}{(T+1)^2}+\frac{16}{3} \sqrt{C\left(\frac{8}{5}, \frac{4}{5}\right)} \frac{\sigma_g \gamma_0}{(T+1)^{2 / 5}}+\frac{16}{3} C\left(\frac{6}{5}, \frac{4}{5}\right) \frac{L_h \gamma_0^3}{(T+1)^{2 / 5}} \\
& \quad+\frac{L_g \gamma_0^2}{2} \frac{1}{T+1}+\frac{\gamma_0 \varepsilon^{\prime}}{3}+\frac{4 \gamma_0 D_g}{3} \gamma^H+\frac{16 \gamma_0}{3} D_g \gamma^H C\left(0, \frac{4}{5}\right)(T+1)^{4 / 5}+\frac{8 \gamma_0^2}{3} D_h \gamma^H C\left(1, \frac{4}{5}\right) \frac{1}{(T+1)^{1 / 5}} .
\end{aligned}
$$
となる。

#### 定理1におけるサンプル複雑度の導出
定理1から、
$$
\mathcal{O}\left(\frac{\sigma_g + L_h}{(T+1)^{2/5}}\right) \leq \varepsilon
$$
よって、

$$
\frac{1}{(T+1)^{2/5}} \leq C \varepsilon
$$
となる。

$$
(T+1)^{2/5} \geq \frac{1}{C\varepsilon}
$$

両辺を $5/2$ 乗すると、

$$
T+1 \geq \frac{1}{C^{5/2}\varepsilon^{5/2}}
$$

したがって、

$$
T = \mathcal{O}(\varepsilon^{-5/2}) = \mathcal{O}(\varepsilon^{-2.5})
$$

となる。

#### 定理2(HARPGのグローバル収束)
仮定1、仮定3、仮定4が成り立つとする。$\gamma_t = \gamma_0 \eta_t^{1/2}$, $\eta_t = \frac{2}{t+2}$ とし、ここで $\gamma_0 = \min \left\{ \frac{1}{8\sqrt{6}(L_g + \sigma_g + D_h \gamma^H)}, \frac{\sqrt{2} M_g}{\sqrt{3} \sigma_g \mu_F} \right\}$, $H = (1-\gamma)^{-1} \log(T+1)$ とする。このとき、任意の $T \geq 1$ に対して、以下が成り立つ。

$$
J^* - \mathbb{E}[J(\theta_T)] \leq \mathcal{O}\left( \frac{\sigma_g + L_g + \sigma_h}{\sqrt{T+1}} \right) + \frac{2\sqrt{\varepsilon_{\text{bias}}}}{1-\gamma}
$$



#### 定理3((N)-HARPGのグローバル収束)
仮定1、仮定3、仮定4が成り立つとする。$\gamma_t = \frac{6M_g}{\mu_F(t+2)}$, $\eta_t = \frac{2}{t+2}$, $H = (1-\gamma)^{-1} \log(T+1)$ とする。このとき、任意の $T \geq 1$ に対して、以下が成り立つ。

$$
J^* - \mathbb{E}[J(\theta_T)] \leq \mathcal{O}\left( \frac{\sigma_g + L_g + \sigma_h}{\sqrt{T+1}} \right) + \frac{\sqrt{\varepsilon_{\text{bias}}}}{1-\gamma}
$$
